In [38]:
import pandas as pd
import numpy as np

In [39]:
train = pd.read_csv("./train_cleaned_unfiltered.csv")
list_classes = ["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train = train[list_classes]

train_files = ["out/train_1703_FG_LDA.csv", 
               "out/train_1703_FG_lstm_combined.csv", 
               "out/train_1703_FG_nb_lr_combined.csv",
               "out/train_1703_FG_nn_combined.csv",
               "out/train_1703_FG_rnd_forest_combined.csv",
               "out/train_1703_FG_xgboost_combined.csv",
               "out/train_1803_FG_LSTMmulticlass.csv",
               "out/train_1803_FG_vader.csv"
              ]

for f in train_files:
    table =  pd.read_csv(f, header=None)
    table.rename(columns={0:'id'}, inplace=True)
    train = pd.merge(train, table, on = ["id"])
    print(train.shape)

(159571, 17)
(159571, 24)
(159571, 31)
(159571, 38)
(159571, 45)
(159571, 52)
(159571, 58)
(159571, 62)


In [40]:
train.isnull().values.any()

False

In [41]:
test = pd.read_csv("./test_cleaned_unfiltered.csv")
list_classes = ["id"]
test = test[list_classes]

test_files =  ["out/test_1703_FG_LDA.csv", 
               "out/test_1703_FG_lstm_combined.csv", 
               "out/test_1703_FG_nb_lr_combined.csv",
               "out/test_1703_FG_nn_combined.csv",
               "out/test_1703_FG_rnd_forest_combined.csv",
               "out/test_1703_FG_xgboost_combined.csv",
               "out/test_1803_FG_LSTMmulticlass.csv",
               "out/test_1803_FG_vader.csv",
               "out/test_kevin.csv"
              ]

for f in test_files:
    table =  pd.read_csv(f, header=None)
    table.rename(columns={0:'id'}, inplace=True)
    test = pd.merge(test, table, on = ["id"])
    print(test.shape)

(153164, 11)
(153164, 18)
(153164, 25)
(153164, 32)
(153164, 39)
(153164, 46)
(153164, 52)
(153164, 56)
(153164, 62)


In [42]:
test.isnull().values.any()

False

In [43]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_classes = ["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

X = train.drop(list_classes, axis= 1 ).values

In [44]:
tmp =  pd.read_csv("out/train_kevin.csv")
X_train_kevin = tmp.values
X = np.column_stack([X, X_train_kevin])

In [45]:
X.shape

(159571, 61)

In [8]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression


class LR:
    
    def __init__(self):
        self.models = []

    def get_mdl(self, X, y):
        m = LogisticRegression(C=10, dual=True)
        return m.fit(X, y)
    
    def fit(self, X, y):
        if len(y.shape)>1:
            for i in range(y.shape[1]):
                m = self.get_mdl(X, y[:,i])
                self.models.append((m))
        else:
            m = self.get_mdl(X, y)
            self.models.append((m))
        
    def predict(self, X):
        preds = np.zeros((X.shape[0], len(self.models)))
        for i in range(len(self.models)):
            m = self.models[i]
            preds[:,i] = m.predict_proba(X)[:,1]
        return preds

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

for tbl in [X_train, X_val, y_train, y_val]:
    print tbl.shape

(106912, 55)
(52659, 55)
(106912, 6)
(52659, 6)


In [11]:
l = LogisticRegression(C=10, dual=True)
lr = LR()
lr.fit(X, y)
pred = lr.predict(X)
print(roc_auc_score(y, pred))

0.9999851332423976


In [12]:
to_predict = test.drop(["id"], axis= 1 ).values
to_predict.shape
y_pred = lr.predict(to_predict)

In [ ]:
y_pred.shape

In [10]:
l = LogisticRegression(C=10, dual=True)
lr = LR()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
print(roc_auc_score(y_val, pred))

0.9999873282424735


In [13]:

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

submid = pd.DataFrame({'id': test["id"]})
submission = pd.concat([submid, pd.DataFrame(y_pred, columns = list_classes)], axis=1)
print(submission.shape)
submission.to_csv('submission.csv', index=False)



(153164, 7)


In [18]:
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from  keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

class nn_1vsall_clf:
    
        
    def __init__(self):
        self.models = []
    
    def get_mdl(self, X, y):
        adam = Adam(lr=0.001)
        model = Sequential()
        model.add(BatchNormalization(input_shape=(X.shape[1],)))
        model.add(Dense(2048, activation='sigmoid'))
        model.add(Dropout(rate=0.6))
        model.add(Dense(2048, activation='sigmoid'))
        model.add(Dropout(rate=0.6))
        #self.model.add(Dense(2048, activation='sigmoid'))
        #self.model.add(Dropout(rate=0.6))
        #self.model.add(Dense(2048, activation='sigmoid'))
        #self.model.add(Dropout(rate=0.6))
        if len(y.shape) > 1:
            model.add(Dense(y.shape[1], activation='sigmoid'))
            model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])
        else:
            model.add(Dense(1, activation='sigmoid'))
            model.compile(loss="roc_auc_score ", optimizer=adam, metrics=["accuracy"])

        #print(self.model.summary())
        batch_size = 512
        model.fit(X, y, batch_size=512, epochs=20)
        return model

    def fit(self, X, y):
        if len(y.shape)>1:
            for i in range(y.shape[1]):
                m = self.get_mdl(X, y[:,i])
                self.models.append((m))
        else:
            m = self.get_mdl(X, y)
            self.models.append((m))
        
    def predict(self, X):
        preds = np.zeros((X.shape[0], len(self.models)))
        for i in range(len(self.models)):
            m = self.models[i]
            preds[:,i] = m.predict(X).flatten()
        return preds

In [19]:
nn = nn_1vsall_clf()
nn.fit(X_train, y_train)
pred = nn.predict(X_val)
print(roc_auc_score(y_val, pred))

ValueError: Unknown loss function:roc_auc_score 

In [51]:
import tflearn
tf.reset_default_graph()

# Building deep neural network
input_layer = tflearn.input_data(shape=[None, 61])
dense1 = tflearn.fully_connected(input_layer, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout1 = tflearn.dropout(dense1, 0.8)
dense2 = tflearn.fully_connected(dropout1, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout2 = tflearn.dropout(dense2, 0.8)
softmax = tflearn.fully_connected(dropout2, 6, activation='sigmoid')

# Regression using SGD with learning rate decay and Top-3 accuracy
sgd = tflearn.Adam(learning_rate=0.0001)
top_k = tflearn.metrics.Top_k(3)
net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,
                         loss='roc_auc_score')

# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X, y, n_epoch=20, run_id="dense_model")



Training Step: 49879  | total loss: 1.64143 | time: 14.700s
| Adam | epoch: 020 | loss: 1.64143 -- iter: 159552/159571
Training Step: 49880  | total loss: 1.47887 | time: 14.706s
| Adam | epoch: 020 | loss: 1.47887 -- iter: 159571/159571
--


In [52]:
to_predict = test.drop(["id"], axis= 1 ).values
to_predict.shape
y_pred = model.predict(to_predict)

In [53]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

submid = pd.DataFrame({'id': test["id"]})
submission = pd.concat([submid, pd.DataFrame(y_pred, columns = list_classes)], axis=1)
print(submission.shape)
submission.to_csv('submission-tflearn.csv', index=False)

(153164, 7)


In [87]:
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from  keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

class nn_tf:
    
        
    def __init__(self):
        self.models = []
    
    def get_mdl(self, X, y):
       # Building deep neural network
        tf.reset_default_graph()

        input_layer = tflearn.input_data(shape=[None, 61])
        dense1 = tflearn.fully_connected(input_layer, 256, activation='relu',
                                         regularizer='L2', weight_decay=0.001)
        dropout1 = tflearn.dropout(dense1, 0.8)
        dense2 = tflearn.fully_connected(dropout1, 256, activation='relu',
                                         regularizer='L2', weight_decay=0.001)
        dropout2 = tflearn.dropout(dense2, 0.8)
        softmax = tflearn.fully_connected(dropout2, 1, activation='sigmoid')

        # Regression using SGD with learning rate decay and Top-3 accuracy
        sgd = tflearn.Adam(learning_rate=0.0001)
        top_k = tflearn.metrics.Top_k(3)
        net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,
                                 loss='roc_auc_score')

        # Training
        model = tflearn.DNN(net, tensorboard_verbose=0)
        model.fit(X, y, n_epoch=1, run_id="dense_model")
        return model

    def fit(self, X, y):
        if len(y.shape)>1:
            for i in range(y.shape[1]):
                m = self.get_mdl(X, y[:,1].reshape(y.shape[0],1))
                self.models.append((m))
        else:
            m = self.get_mdl(X, y)
            self.models.append((m))
        
    def predict(self, X):
        preds = np.zeros((X.shape[0], len(self.models)))
        for i in range(len(self.models)):
            m = self.models[i]
            preds[:,i] = m.predict(X).flatten()
        return preds

In [84]:
y[:,1].reshape(y.shape[0],1)

array([0])

In [88]:
nn = nn_tf()
nn.fit(X, y)

Training Step: 2493  | total loss: 0.00030 | time: 15.514s
| Adam | epoch: 001 | loss: 0.00030 -- iter: 159552/159571
Training Step: 2494  | total loss: 0.00027 | time: 15.519s
| Adam | epoch: 001 | loss: 0.00027 -- iter: 159571/159571
--


In [89]:
nn.models

In [90]:
y_pred = nn.predict(to_predict)

In [91]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

submid = pd.DataFrame({'id': test["id"]})
submission = pd.concat([submid, pd.DataFrame(y_pred, columns = list_classes)], axis=1)
print(submission.shape)
submission.to_csv('submission-tflearn.csv', index=False)

(153164, 7)
